# Advent of Code 2025 - Day 1

[Day 1: Secret Entrance](https://adventofcode.com/2025/day/1)

In [ ]:
%use dataframe

In [ ]:
data class Row(public val instruction: String) {}

val input = DataFrame.read("./input.txt", listOf("instruction")) as DataFrame<Row>

Map the instructions in the input format "L32" or "R60" to -32 and 60 respectively so that we can easily use them in addition

In [ ]:
val instructions = input.map {
    val num = it.instruction.substring(1).toInt()
    when (it.instruction[0]) {
        'R' -> num * 1
        'L' -> num * -1
        else -> throw RuntimeException("Unknown direction")
    }
}

## Part 1


Fold the instructions with the initial position, keeping a running list. The first parameter to `runningFold` is the initial position of the dial

In [ ]:
val initialDialPosition = 50
val rolloverPointMaxExcl = 100

val positions = instructions.runningFold(initialDialPosition) { acc, inst ->
    var newPos = (acc + inst) % rolloverPointMaxExcl
    if (newPos < 0) {
        newPos += rolloverPointMaxExcl
    }
    newPos
}
/**
 * Output to notebook for validation
 */
positions.toDataFrame()

Here's the password:

In [ ]:
positions.filter { it == 0 }.count()

## Part 2

In [ ]:
data class PositionWithCrossings(val prevPos: Int?, val instruction: Int?, val position: Int, val crossingCount: Int)

val instructionsWithPosition = instructions.zip(positions.drop(1))

val positionsWithCrossings = instructionsWithPosition.runningFold(
    PositionWithCrossings(null, null, initialDialPosition, 0)
) { prev, (inst, pos) ->
    // Simplify instructions with distance of more than 100
    val instructionCrossings = inst / rolloverPointMaxExcl
    val instructionMove = inst % rolloverPointMaxExcl
    // Get unbounded new position (may be more than 100 or less than 0)
    val unboundedPosition = (prev.position + instructionMove)

    var crossings = Math.abs(instructionCrossings)
    // If we moved from anywhere but the 0 position and crossed the 0 point (by having ended up out of bounds)
    if (prev.position != 0 && (unboundedPosition >= 100 || unboundedPosition < 0)) {
        crossings++;
    // Or if we ended on 0 and moved at all
    } else if (pos == 0 && instructionMove != 0) {
        crossings++;
    }

    PositionWithCrossings(prev.position, inst, pos, crossings)
}
/**
 * Output to notebook for validation
 */
positionsWithCrossings.toDataFrame()

Here's the password:

In [ ]:
positionsWithCrossings.fold(0) { acc, pos -> acc + pos.crossingCount}